# 11で出した重要語と共起する単語を算出

In [17]:
ID = "UT-sei"

POPULATION_ID = "population"

#10の途中経過
without_stopwords_text_file = "Progresses/NonStopword/%s-from-%s.txt"%(ID, POPULATION_ID)

#10の結果
sentiment_file = "Sentiment/%s-from-%s.txt"%(ID, POPULATION_ID)

##TFIDFモデル
tfidf_model_file = "Models/Tfidf/%s-from-%s.model"%(ID, POPULATION_ID)

#内部的に品詞を区別する区切り文字
TOKEN_DIVIDER = "<334>"

In [18]:
#ここだけのストップワード
#投稿そのものを除外するのではなく、関連単語から除外する
local_stopwords = ["東大", "br"]

## 読み込み

In [19]:
import pandas as pd
import seaborn as sns

In [20]:
df_main = pd.read_table(without_stopwords_text_file)

#本ノートでは生テキストではなく形態素解析後のテキストが渡される
def tokenize(text):
    return text.split(TOKEN_DIVIDER)

df_main.head(2)

,date,link_index,number,content,disassembled_all,disassembled_target,Sentiment
0,2017/10/30|23/00/00,18471,19,東京大学 新卒就職先トップ１０ ＜２０１１年＞ ＜２０１２年＞ ...,*<334>トウキョウ<334>大学<334> <334>新卒<334>就職<334>先<3...,トウキョウ<334>大学<334>新卒<334>就職<334>先<334>トップ-top<3...,pos
1,2009/01/07|23/05/00,4800,211,■東大生の選んだ法科大学院ランキング■ http://changi.2ch.net/te...,*<334>■<334>東大<334>生<334>の<334>選ぶ<334>た<334>法科...,東大<334>法科<334>大学<334>ランキング-ranking<334>http<33...,neu


In [21]:
df_sentiment = pd.read_table(sentiment_file)

df_sentiment.head(2)

,Token,TFIDF,Sentiment,Counts
0,東大,0.115418,-0.639404,11811
1,無い,0.037442,-0.763441,5394


## 単純な共起回数

### 単語ベクトルの用意

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


In [23]:
vectorizer = CountVectorizer(tokenizer=tokenize, stop_words=local_stopwords, ngram_range=(1,1), min_df=20)
vectorizer = vectorizer.fit(df_main["disassembled_target"])

c:\Users\brigh\anaconda3\envs\jpnlp\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [24]:
#keywordに共起する回数の多いものを数える
def get_freq(keyword):
    #keywordを含む行を走査
    df_keyword = np.empty((0, df_main.shape[1]))
    for _, row in df_main.iterrows():
        if keyword in tokenize(row["disassembled_all"]):
            df_keyword = np.vstack([df_keyword, row.values])

    #pandas行列に
    df_keyword = pd.DataFrame(df_keyword, columns=df_main.columns)

    #単語ベクトルの回数数える
    freq_spycy = vectorizer.transform(df_keyword["disassembled_target"])
    freq_df = pd.DataFrame(freq_spycy.toarray(), columns=vectorizer.get_feature_names_out())
    freq_sum = freq_df.sum(axis=0)
    
    #単語名と結び付け
    freq_sum = freq_sum.sort_values(ascending=False)

    #ストップワードの削除
    for word in local_stopwords:
        try:
            freq_sum = freq_sum.drop(word)
        except:
            pass

    return freq_sum

get_freq("良い").head(5)

大学           3642
良い           3268
無い           3250
マーチ-march    2020
未満           1945
dtype: int64

In [25]:
df_sentiment["CoOccurre-FREQ"] = ""
for index, row in df_sentiment.iterrows():
    freq = get_freq(row["Token"])
    end = 6
    if len(freq) <= 1:
        end = 1
    elif len(freq) < 6:
        end = len(freq)

    if end != 1:
        cooc = ", ".join(list(freq.iloc[1:end].index))
        df_sentiment.at[index, "CoOccurre-FREQ"] = cooc

pd.set_option('display.max_rows', 100)
df_sentiment.head(100).style.background_gradient(cmap="vlag_r", axis=0, subset="Sentiment")

,Token,TFIDF,Sentiment,Counts,CoOccurre-FREQ
0,東大,0.115418,-0.639404,11811,"無い, 事, 良い, 部, マーチ-march"
1,無い,0.037442,-0.763441,5394,"大学, 事, マーチ-march, 未満, 良い"
2,事,0.028268,-0.694920,3740,"無い, 大学, マーチ-march, 未満, 良い"
3,大学,0.028167,-0.612665,3253,"無い, 事, マーチ-march, 未満, 明治"
4,ｗ,0.021295,-0.525586,1876,"無い, 事, 大学, 良い, 部"
5,良い,0.018640,-0.596716,2497,"良い, 無い, マーチ-march, 未満, 事"
6,学歴,0.016818,-0.627374,1527,"学歴, 無い, 未満, マーチ-march, march"
7,奴,0.016664,-0.809903,1757,"奴, 大学, 未満, マーチ-march, march"
8,京大,0.015152,-0.648760,1210,"大学, 無い, 事, 慶応, 部"
9,部,0.014669,-0.549745,1568,"大学, 無い, 医学, 年, 事"


## 分析対象内

## 母集団に対するTFIDF

### 母集団TFIDFモデルをサルベージ

In [26]:
import pickle
from copy import deepcopy

with open(tfidf_model_file, "rb") as f:
    tfidf_model = pickle.load(f)

#対象（名詞・形容詞）を見せる用
tfidf_model_showing = deepcopy(tfidf_model)
tfidf_model_showing.ngram = (1,1)
tfidf_model_showing.tokenizer = tokenize

### TFIDFを計算

In [27]:
#keywordに共起する回数の多いものを数える
def get_tfidf(keyword):
    #keywordを含む行を走査
    df_keyword = np.empty((0, df_main.shape[1]))
    for _, row in df_main.iterrows():
        if keyword in tokenize(row["disassembled_all"]):
            df_keyword = np.vstack([df_keyword, row.values])

    #pandas行列に
    df_keyword = pd.DataFrame(df_keyword, columns=df_main.columns)

    if df_keyword.shape[0] == 0:
        return pd.Series()

    #TFIDF計算
    tfidf_spycy = tfidf_model_showing.transform(df_keyword["disassembled_target"])
    tfidf_df = pd.DataFrame(tfidf_spycy.toarray(), columns=tfidf_model_showing.get_feature_names_out())
    tfidf_mean = tfidf_df.mean(axis=0)
    
    #単語名と結び付け
    tfidf_mean = tfidf_mean.sort_values(ascending=False)

    #ストップワードの削除
    for word in local_stopwords:
        try:
            tfidf_mean = tfidf_mean.drop(word)
        except:
            pass

    return tfidf_mean

get_tfidf("良い").head(5)

良い      0.126139
無い      0.051443
大学      0.041283
事       0.038492
頭 良い    0.031860
dtype: float64

In [28]:
df_sentiment["CoOccurre-TFIDF"] = ""
for index, row in df_sentiment.iterrows():
    freq = get_tfidf(row["Token"])
    end = 6
    if len(freq) <= 1:
        end = 1
    elif len(freq) < 6:
        end = len(freq)

    if end != 1:
        cooc = ", ".join(list(freq.iloc[1:end].index))
        df_sentiment.at[index, "CoOccurre-TFIDF"] = cooc

pd.set_option('display.max_rows', 200)
df_sentiment.head(200).style.background_gradient(cmap="vlag_r", axis=0, subset="Sentiment")

C:\Users\brigh\AppData\Local\Temp\ipykernel_42712\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\brigh\AppData\Local\Temp\ipykernel_42712\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\brigh\AppData\Local\Temp\ipykernel_42712\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\brigh\AppData\Local\Temp\ipykernel_42712\3496934589.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()


,Token,TFIDF,Sentiment,Counts,CoOccurre-FREQ,CoOccurre-TFIDF
0,東大,0.115418,-0.639404,11811,"無い, 事, 良い, 部, マーチ-march","大学, 事, 良い, 京大, 学歴"
1,無い,0.037442,-0.763441,5394,"大学, 事, マーチ-march, 未満, 良い","事, 大学, 無い 東大, 良い, 学歴"
2,事,0.028268,-0.694920,3740,"無い, 大学, マーチ-march, 未満, 良い","無い, 大学, 事 無い, 良い, 学歴"
3,大学,0.028167,-0.612665,3253,"無い, 事, マーチ-march, 未満, 明治","無い, 事, トウキョウ, ワセダ, 部"
4,ｗ,0.021295,-0.525586,1876,"無い, 事, 大学, 良い, 部","無い, 無い ｗ, 事, ｗ 東大, 大学"
5,良い,0.018640,-0.596716,2497,"良い, 無い, マーチ-march, 未満, 事","無い, 大学, 事, 頭 良い, 頭"
6,学歴,0.016818,-0.627374,1527,"学歴, 無い, 未満, マーチ-march, march","無い, 大学, 事, 良い, 奴"
7,奴,0.016664,-0.809903,1757,"奴, 大学, 未満, マーチ-march, march","無い, 大学, 事, 多い, 奴 多い"
8,京大,0.015152,-0.648760,1210,"大学, 無い, 事, 慶応, 部","東大 京大, 大学, 無い, 事, 阪大"
9,部,0.014669,-0.549745,1568,"大学, 無い, 医学, 年, 事","医学, 医学 部, 大学, 法学, 法学 部"
